<a href="https://colab.research.google.com/github/purvasingh96/Deep-Reinforcement-Learning/blob/master/4.%20Deep%20Q%20Networks/Credit_Card_Fraud_Detection_via_DQNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading data and Making *gym-fraud-detection* environment

In [0]:
import shutil

shutil.rmtree('/content/gym-fraud-detection')

In [0]:
!unzip gym-fraud-detection.zip

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
unzip:  cannot find or open gym-fraud-detection.zip, gym-fraud-detection.zip.zip or gym-fraud-detection.zip.ZIP.


In [0]:
os.chdir('/content/')

In [10]:
print(os.getcwd())

/content


In [0]:
!unzip gym-fraud-detection.zip

In [0]:
import os
os.chdir('gym-fraud-detection')

In [0]:
pip install -e .

In [0]:
import gym
import gym_fraud_detection

In [0]:
env = gym.make('gym-fraud-detection-v0')

# Download proper libraries

In [0]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


In [0]:
# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")